In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
train_file = "./train.csv"
test_file = "./test.csv"
num_cols = ["ps_reg_01", "ps_reg_02", "ps_reg_03","ps_car_12", "ps_car_13", "ps_car_14", "ps_car_15"]
ignore_cols = ["id", "target", "ps_calc_01", "ps_calc_02", "ps_calc_03", "ps_calc_04", "ps_calc_05", 
               "ps_calc_06", "ps_calc_07", "ps_calc_08", "ps_calc_09", "ps_calc_10", "ps_calc_11", 
               "ps_calc_12", "ps_calc_13", "ps_calc_14","ps_calc_15_bin", "ps_calc_16_bin", 
               "ps_calc_17_bin","ps_calc_18_bin", "ps_calc_19_bin", "ps_calc_20_bin"]

In [3]:
cfg = {
    "feature_size": None,
    "field_size": None,
    "embed_size":128,
    "deep_nn":[256,256],
    "dropout_fm": 0,
    "dropout_deep": 0.2,
    "epoch":200,
    "batch":10000,
    "split": 0.2,
    "class_weight": None 
}

In [4]:
def overview(cfg):    
    dfTrain = pd.read_csv(train_file)
    dfTest = pd.read_csv(test_file)
    df = pd.concat([dfTrain,dfTest], sort=False)

    field_size = len(df.columns) - len(ignore_cols)
    feature_dict = {}
    feature_size = 0
    for col in df.columns:
        if col in ignore_cols:
            continue
        elif col in num_cols:
            feature_dict[col] = feature_size
            feature_size += 1
        else:
            unique_val = df[col].unique()
            feature_dict[col] = dict(zip(unique_val,range(feature_size,len(unique_val) + feature_size)))
            feature_size += len(unique_val)
    
    cfg['field_size'] = field_size
    cfg['feature_size'] = feature_size
    return dfTrain, feature_dict

In [5]:
dfTrain, feature_dict = overview(cfg)

In [6]:
def preprocess(train_df, cfg):
    label_df = train_df[['target']]
    neg, pos = np.bincount(label_df.values.flatten())
    cfg["class_weight"] = {0: (1/neg)*(neg+pos), 1: (1/pos)*(neg+pos)}
    
    train_df.drop(['target','id'],axis=1,inplace=True)
    feature_idx = train_df.copy()
    feature_val = train_df.copy()
    for col in feature_idx.columns:
        if col in ignore_cols:
            feature_idx.drop(col,axis=1,inplace=True)
            feature_val.drop(col,axis=1,inplace=True)
            continue
        elif col in num_cols:
            feature_idx[col] = feature_dict[col]
        else:
            feature_idx[col] = feature_idx[col].map(feature_dict[col])
            feature_val[col] = 1        
            
    split_idx = feature_idx.shape[0] - round(feature_idx.shape[0]*cfg["split"])
    train_input = [feature_idx[:split_idx].values, feature_val[:split_idx].values]
    train_y = label_df[:split_idx].values
    validate_input = [feature_idx[split_idx:].values, feature_val[split_idx:].values]
    validate_y = label_df[split_idx:].values

    return train_input, train_y, validate_input, validate_y

In [7]:
train_input, train_y, validate_input, validate_y = preprocess(dfTrain, cfg)

In [8]:
class DeepFM(tf.keras.Model):
    def __init__(self, cfg):
        super(DeepFM, self).__init__()
        self.feature_size = cfg['feature_size']
        self.field_size = cfg['field_size']
        self.embed_size = cfg['embed_size']
        self.deep_nn = cfg['deep_nn']
        
        self.dropout_fm = cfg['dropout_fm']
        self.dropout_deep = cfg['dropout_deep']
        
        # fm        
        self.feature_weight = tf.keras.layers.Embedding(cfg['feature_size'], 1)
        self.feature_embed = tf.keras.layers.Embedding(cfg['feature_size'], cfg['embed_size'])

        # dnn
        for layer in range(len(cfg['deep_nn'])):
            setattr(self, 'dense_' + str(layer), tf.keras.layers.Dense(self.deep_nn[layer]))
            setattr(self, 'batchNorm_' + str(layer), tf.keras.layers.BatchNormalization())
            setattr(self, 'activation_' + str(layer), tf.keras.layers.Activation('relu'))
            setattr(self, 'dropout_' + str(layer), tf.keras.layers.Dropout(self.dropout_deep))
            
        self.fc = tf.keras.layers.Dense(1, activation='sigmoid', use_bias=True)

    def call(self, inputs, training=True):
        # inputs = [feature_idx, feature_val]
        reshaped_feature_val = tf.cast(tf.reshape(inputs[1], shape=[-1,self.field_size,1]), tf.float32)
        # linear        
        weights = self.feature_weight(inputs[0])
        linear = tf.reduce_sum(tf.multiply(weights,reshaped_feature_val),2)
        
        # fm  
        embeddings = self.feature_embed(inputs[0])
        second_inner = tf.multiply(embeddings,reshaped_feature_val)
        
        summed_features_emb = tf.reduce_sum(second_inner,1)
        summed_features_emb_square = tf.square(summed_features_emb)
        
        squared_features_emb = tf.square(second_inner)
        squared_sum_features_emb = tf.reduce_sum(squared_features_emb,1)
        
        fm = 0.5 * tf.subtract(summed_features_emb_square,squared_sum_features_emb)
        
        # dnn
        y_deep = tf.reshape(embeddings,shape=[-1,self.field_size * self.embed_size])
        for layer in range(0, len(self.deep_nn)):
            y_deep = getattr(self, 'dense_' + str(layer))(y_deep)
            y_deep = getattr(self, 'batchNorm_' + str(layer))(y_deep, training=training)
            y_deep = getattr(self, 'activation_' + str(layer))(y_deep)
            y_deep = getattr(self, 'dropout_' + str(layer))(y_deep, training=training)
            
        # concat
        concat = tf.concat([linear, fm, y_deep], axis=1)                                
        out = self.fc(concat)
        return out

In [9]:
cfg

{'feature_size': 257,
 'field_size': 37,
 'embed_size': 128,
 'deep_nn': [256, 256],
 'dropout_fm': 0,
 'dropout_deep': 0.2,
 'epoch': 200,
 'batch': 10000,
 'split': 0.2,
 'class_weight': {0: 1.0378261885415976, 1: 27.43671061122891}}

In [10]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='bin_acc'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

In [11]:
model = DeepFM(cfg)
# model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.compile(optimizer = 'adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=METRICS)
model.fit(train_input, train_y, epochs=cfg['epoch'], batch_size=cfg['batch'], shuffle=True, class_weight=cfg['class_weight'],
          verbose=1, validation_data=(validate_input, validate_y))

Train on 476170 samples, validate on 119042 samples
Epoch 1/200
476170/476170 [==============================] - 8s 17us/sample - loss: 1.3725 - tp: 5159.0000 - fp: 78303.0000 - tn: 380526.0000 - fn: 12182.0000 - bin_acc: 0.8100 - precision: 0.0618 - recall: 0.2975 - auc: 0.5999 - val_loss: 1.3901 - val_tp: 1.0000 - val_fp: 3.0000 - val_tn: 114686.0000 - val_fn: 4352.0000 - val_bin_acc: 0.9634 - val_precision: 0.2500 - val_recall: 2.2973e-04 - val_auc: 0.5542
Epoch 2/200
476170/476170 [==============================] - 6s 12us/sample - loss: 1.3562 - tp: 4079.0000 - fp: 49155.0000 - tn: 409674.0000 - fn: 13262.0000 - bin_acc: 0.8689 - precision: 0.0766 - recall: 0.2352 - auc: 0.6261 - val_loss: 1.3807 - val_tp: 12.0000 - val_fp: 62.0000 - val_tn: 114627.0000 - val_fn: 4341.0000 - val_bin_acc: 0.9630 - val_precision: 0.1622 - val_recall: 0.0028 - val_auc: 0.6167
Epoch 3/200
476170/476170 [==============================] - 6s 12us/sample - loss: 1.3531 - tp: 4395.0000 - fp: 51547.0000 - 

Epoch 21/200
476170/476170 [==============================] - 6s 13us/sample - loss: 1.3068 - tp: 6960.0000 - fp: 54640.0000 - tn: 404189.0000 - fn: 10381.0000 - bin_acc: 0.8635 - precision: 0.1130 - recall: 0.4014 - auc: 0.6689 - val_loss: 1.3677 - val_tp: 640.0000 - val_fp: 7524.0000 - val_tn: 107165.0000 - val_fn: 3713.0000 - val_bin_acc: 0.9056 - val_precision: 0.0784 - val_recall: 0.1470 - val_auc: 0.5770
Epoch 22/200
476170/476170 [==============================] - 6s 13us/sample - loss: 1.3050 - tp: 6988.0000 - fp: 53764.0000 - tn: 405065.0000 - fn: 10353.0000 - bin_acc: 0.8653 - precision: 0.1150 - recall: 0.4030 - auc: 0.6707 - val_loss: 1.3681 - val_tp: 984.0000 - val_fp: 13012.0000 - val_tn: 101677.0000 - val_fn: 3369.0000 - val_bin_acc: 0.8624 - val_precision: 0.0703 - val_recall: 0.2261 - val_auc: 0.5871
Epoch 23/200
476170/476170 [==============================] - 7s 14us/sample - loss: 1.3034 - tp: 7094.0000 - fp: 54383.0000 - tn: 404446.0000 - fn: 10247.0000 - bin_acc: 

Epoch 41/200
476170/476170 [==============================] - 6s 13us/sample - loss: 1.2768 - tp: 7909.0000 - fp: 47593.0000 - tn: 411236.0000 - fn: 9432.0000 - bin_acc: 0.8802 - precision: 0.1425 - recall: 0.4561 - auc: 0.6993 - val_loss: 1.3722 - val_tp: 972.0000 - val_fp: 13469.0000 - val_tn: 101220.0000 - val_fn: 3381.0000 - val_bin_acc: 0.8585 - val_precision: 0.0673 - val_recall: 0.2233 - val_auc: 0.5787
Epoch 42/200
476170/476170 [==============================] - 6s 13us/sample - loss: 1.2750 - tp: 7937.0000 - fp: 46442.0000 - tn: 412387.0000 - fn: 9404.0000 - bin_acc: 0.8827 - precision: 0.1460 - recall: 0.4577 - auc: 0.7000 - val_loss: 1.3744 - val_tp: 1140.0000 - val_fp: 16740.0000 - val_tn: 97949.0000 - val_fn: 3213.0000 - val_bin_acc: 0.8324 - val_precision: 0.0638 - val_recall: 0.2619 - val_auc: 0.5803
Epoch 43/200
476170/476170 [==============================] - 6s 13us/sample - loss: 1.2735 - tp: 8010.0000 - fp: 46523.0000 - tn: 412306.0000 - fn: 9331.0000 - bin_acc: 0.

Epoch 61/200
476170/476170 [==============================] - 6s 13us/sample - loss: 1.2600 - tp: 8457.0000 - fp: 44128.0000 - tn: 414701.0000 - fn: 8884.0000 - bin_acc: 0.8887 - precision: 0.1608 - recall: 0.4877 - auc: 0.7179 - val_loss: 1.3726 - val_tp: 823.0000 - val_fp: 11172.0000 - val_tn: 103517.0000 - val_fn: 3530.0000 - val_bin_acc: 0.8765 - val_precision: 0.0686 - val_recall: 0.1891 - val_auc: 0.5681
Epoch 62/200
476170/476170 [==============================] - 6s 12us/sample - loss: 1.2580 - tp: 8495.0000 - fp: 42824.0000 - tn: 416005.0000 - fn: 8846.0000 - bin_acc: 0.8915 - precision: 0.1655 - recall: 0.4899 - auc: 0.7190 - val_loss: 1.3814 - val_tp: 1176.0000 - val_fp: 18598.0000 - val_tn: 96091.0000 - val_fn: 3177.0000 - val_bin_acc: 0.8171 - val_precision: 0.0595 - val_recall: 0.2702 - val_auc: 0.5791
Epoch 63/200
476170/476170 [==============================] - 6s 13us/sample - loss: 1.2577 - tp: 8457.0000 - fp: 42361.0000 - tn: 416468.0000 - fn: 8884.0000 - bin_acc: 0.

Epoch 81/200
476170/476170 [==============================] - 6s 12us/sample - loss: 1.2459 - tp: 8889.0000 - fp: 39688.0000 - tn: 419141.0000 - fn: 8452.0000 - bin_acc: 0.8989 - precision: 0.1830 - recall: 0.5126 - auc: 0.7309 - val_loss: 1.3795 - val_tp: 1055.0000 - val_fp: 16353.0000 - val_tn: 98336.0000 - val_fn: 3298.0000 - val_bin_acc: 0.8349 - val_precision: 0.0606 - val_recall: 0.2424 - val_auc: 0.5713
Epoch 82/200
476170/476170 [==============================] - 6s 12us/sample - loss: 1.2469 - tp: 8801.0000 - fp: 39763.0000 - tn: 419066.0000 - fn: 8540.0000 - bin_acc: 0.8986 - precision: 0.1812 - recall: 0.5075 - auc: 0.7305 - val_loss: 1.3792 - val_tp: 1064.0000 - val_fp: 16279.0000 - val_tn: 98410.0000 - val_fn: 3289.0000 - val_bin_acc: 0.8356 - val_precision: 0.0614 - val_recall: 0.2444 - val_auc: 0.5703
Epoch 83/200
476170/476170 [==============================] - 5s 12us/sample - loss: 1.2464 - tp: 8824.0000 - fp: 39619.0000 - tn: 419210.0000 - fn: 8517.0000 - bin_acc: 0.

Epoch 101/200
476170/476170 [==============================] - 5s 11us/sample - loss: 1.2390 - tp: 9041.0000 - fp: 37927.0000 - tn: 420902.0000 - fn: 8300.0000 - bin_acc: 0.9029 - precision: 0.1925 - recall: 0.5214 - auc: 0.7395 - val_loss: 1.3755 - val_tp: 792.0000 - val_fp: 11262.0000 - val_tn: 103427.0000 - val_fn: 3561.0000 - val_bin_acc: 0.8755 - val_precision: 0.0657 - val_recall: 0.1819 - val_auc: 0.5629
Epoch 102/200
476170/476170 [==============================] - 6s 12us/sample - loss: 1.2391 - tp: 9024.0000 - fp: 37809.0000 - tn: 421020.0000 - fn: 8317.0000 - bin_acc: 0.9031 - precision: 0.1927 - recall: 0.5204 - auc: 0.7400 - val_loss: 1.3774 - val_tp: 835.0000 - val_fp: 12208.0000 - val_tn: 102481.0000 - val_fn: 3518.0000 - val_bin_acc: 0.8679 - val_precision: 0.0640 - val_recall: 0.1918 - val_auc: 0.5645
Epoch 103/200
476170/476170 [==============================] - 7s 14us/sample - loss: 1.2379 - tp: 9066.0000 - fp: 37299.0000 - tn: 421530.0000 - fn: 8275.0000 - bin_acc:

Epoch 121/200
476170/476170 [==============================] - 6s 13us/sample - loss: 1.2317 - tp: 9278.0000 - fp: 36413.0000 - tn: 422416.0000 - fn: 8063.0000 - bin_acc: 0.9066 - precision: 0.2031 - recall: 0.5350 - auc: 0.7486 - val_loss: 1.3796 - val_tp: 951.0000 - val_fp: 14519.0000 - val_tn: 100170.0000 - val_fn: 3402.0000 - val_bin_acc: 0.8495 - val_precision: 0.0615 - val_recall: 0.2185 - val_auc: 0.5650
Epoch 122/200
476170/476170 [==============================] - 6s 13us/sample - loss: 1.2306 - tp: 9308.0000 - fp: 35976.0000 - tn: 422853.0000 - fn: 8033.0000 - bin_acc: 0.9076 - precision: 0.2055 - recall: 0.5368 - auc: 0.7495 - val_loss: 1.3769 - val_tp: 851.0000 - val_fp: 12315.0000 - val_tn: 102374.0000 - val_fn: 3502.0000 - val_bin_acc: 0.8671 - val_precision: 0.0646 - val_recall: 0.1955 - val_auc: 0.5616
Epoch 123/200
476170/476170 [==============================] - 7s 14us/sample - loss: 1.2305 - tp: 9284.0000 - fp: 35281.0000 - tn: 423548.0000 - fn: 8057.0000 - bin_acc:

Epoch 141/200
476170/476170 [==============================] - 7s 14us/sample - loss: 1.2249 - tp: 9452.0000 - fp: 33790.0000 - tn: 425039.0000 - fn: 7889.0000 - bin_acc: 0.9125 - precision: 0.2186 - recall: 0.5451 - auc: 0.7554 - val_loss: 1.3792 - val_tp: 874.0000 - val_fp: 13275.0000 - val_tn: 101414.0000 - val_fn: 3479.0000 - val_bin_acc: 0.8593 - val_precision: 0.0618 - val_recall: 0.2008 - val_auc: 0.5614
Epoch 142/200
476170/476170 [==============================] - 7s 14us/sample - loss: 1.2242 - tp: 9532.0000 - fp: 34733.0000 - tn: 424096.0000 - fn: 7809.0000 - bin_acc: 0.9107 - precision: 0.2153 - recall: 0.5497 - auc: 0.7549 - val_loss: 1.3793 - val_tp: 850.0000 - val_fp: 12801.0000 - val_tn: 101888.0000 - val_fn: 3503.0000 - val_bin_acc: 0.8630 - val_precision: 0.0623 - val_recall: 0.1953 - val_auc: 0.5636
Epoch 143/200
476170/476170 [==============================] - 6s 13us/sample - loss: 1.2236 - tp: 9479.0000 - fp: 33401.0000 - tn: 425428.0000 - fn: 7862.0000 - bin_acc:

Epoch 161/200
476170/476170 [==============================] - 6s 13us/sample - loss: 1.2190 - tp: 9582.0000 - fp: 31779.0000 - tn: 427050.0000 - fn: 7759.0000 - bin_acc: 0.9170 - precision: 0.2317 - recall: 0.5526 - auc: 0.7608 - val_loss: 1.3809 - val_tp: 863.0000 - val_fp: 13546.0000 - val_tn: 101143.0000 - val_fn: 3490.0000 - val_bin_acc: 0.8569 - val_precision: 0.0599 - val_recall: 0.1983 - val_auc: 0.5613
Epoch 162/200
476170/476170 [==============================] - 6s 13us/sample - loss: 1.2183 - tp: 9683.0000 - fp: 32522.0000 - tn: 426307.0000 - fn: 7658.0000 - bin_acc: 0.9156 - precision: 0.2294 - recall: 0.5584 - auc: 0.7617 - val_loss: 1.3804 - val_tp: 876.0000 - val_fp: 13417.0000 - val_tn: 101272.0000 - val_fn: 3477.0000 - val_bin_acc: 0.8581 - val_precision: 0.0613 - val_recall: 0.2012 - val_auc: 0.5605
Epoch 163/200
476170/476170 [==============================] - 6s 12us/sample - loss: 1.2184 - tp: 9698.0000 - fp: 32955.0000 - tn: 425874.0000 - fn: 7643.0000 - bin_acc:

Epoch 181/200
476170/476170 [==============================] - 6s 14us/sample - loss: 1.2130 - tp: 9881.0000 - fp: 31607.0000 - tn: 427222.0000 - fn: 7460.0000 - bin_acc: 0.9180 - precision: 0.2382 - recall: 0.5698 - auc: 0.7683 - val_loss: 1.3803 - val_tp: 813.0000 - val_fp: 12549.0000 - val_tn: 102140.0000 - val_fn: 3540.0000 - val_bin_acc: 0.8648 - val_precision: 0.0608 - val_recall: 0.1868 - val_auc: 0.5580
Epoch 182/200
476170/476170 [==============================] - 6s 14us/sample - loss: 1.2130 - tp: 9808.0000 - fp: 30707.0000 - tn: 428122.0000 - fn: 7533.0000 - bin_acc: 0.9197 - precision: 0.2421 - recall: 0.5656 - auc: 0.7689 - val_loss: 1.3815 - val_tp: 851.0000 - val_fp: 13184.0000 - val_tn: 101505.0000 - val_fn: 3502.0000 - val_bin_acc: 0.8598 - val_precision: 0.0606 - val_recall: 0.1955 - val_auc: 0.5588
Epoch 183/200
476170/476170 [==============================] - 6s 13us/sample - loss: 1.2132 - tp: 9829.0000 - fp: 31272.0000 - tn: 427557.0000 - fn: 7512.0000 - bin_acc:

In [12]:
predict_for_vali = model.predict(validate_input)

In [13]:
check_pd = pd.concat([pd.DataFrame(predict_for_vali), pd.DataFrame(validate_y)], axis=1)
check_pd.columns = ['predict', 'true']
check_pd.head(40)

,predict,true
0,6.222248e-01,0
1,0.000000e+00,0
2,3.936440e-03,0
3,5.617738e-05,0
4,1.669779e-01,0
5,4.181231e-01,0
6,0.000000e+00,0
7,2.864003e-05,0
8,0.000000e+00,0
9,1.198649e-04,0


In [14]:
sort = check_pd.sort_values(by=["true","predict"],ascending=False)

In [15]:
sort[:200]

,predict,true
21650,1.000000,1
48182,1.000000,1
52771,1.000000,1
64500,1.000000,1
81446,1.000000,1
...,...,...
113761,0.997498,1
68538,0.997455,1
113254,0.997418,1
108988,0.997367,1


In [18]:
pred_sort = check_pd.sort_values(by=["predict","true"],ascending=False)

In [19]:
pred_sort[:200]

,predict,true
21650,1.000000,1
48182,1.000000,1
52771,1.000000,1
64500,1.000000,1
81446,1.000000,1
...,...,...
45880,0.999997,0
85395,0.999997,0
87222,0.999997,0
111700,0.999997,0
